## importing libraries

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing our list of stocks 

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making our first API call

we need to get one-year price returns for each stock in our listings

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

## Parsing our API call 

In [5]:
change_percent = data['year1ChangePercent']

## Executing a Batch API call

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
tickers = []

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price','One-year price return', 'Number of shares to buy']
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_url_call = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url_call).json()
    for symbol in symbol_string.split(','):
        try:
            d = dict(zip(my_columns, [symbol,
                                     data[symbol]['quote']['latestPrice'],
                                     data[symbol]['stats']['year1ChangePercent'],
                                     'N/A']))
            tickers.append(d)
        except:
            continue

final_dataframe = pd.DataFrame(tickers)

## Removing low-momentum stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('One-year price return', ascending=False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop = True, inplace = True)

## Calculating the number of shares to buy

In [8]:
def portofolio_input():
    global val, portofolio_size
    portofolio_size = input('Enter the size of your portofolio: ')
    try:
       val = float(portofolio_size) 
    except:
        print('That is not a number!\n Try again')
        portofolio_size= input('Please enter the size of your portofolio: ')

portofolio_input()
print(portofolio_size)

Enter the size of your portofolio: 10000000
10000000


In [9]:
position_size = val / len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of shares to buy'] = math.floor(position_size/final_dataframe['Price'][i])

## Building a better (and more realisic) momentum strategy

- High-quality momentum stocks show "slow and steady" outperformance over long periods of time
- Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of:
- 1-month price returns
- 3-month price returns
- 6-month price returns
- 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [93]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of shares to buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)
hqm_tickers = []
for symbol_string in symbol_strings:
    batch_api_url_call = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url_call).json()
    for symbol in symbol_string.split(','):
        try:
            d_hqm = dict(zip(hqm_columns, [symbol,
                                           data[symbol]['quote']['latestPrice'],
                                           'N/A',
                                           data[symbol]['stats']['year1ChangePercent'],
                                           'N/A',
                                           data[symbol]['stats']['month6ChangePercent'],
                                           'N/A',
                                           data[symbol]['stats']['month3ChangePercent'],
                                           'N/A',
                                           data[symbol]['stats']['month1ChangePercent'],
                                           'N/A',
                                           'N/A']))
            hqm_tickers.append(d_hqm)
        except:
            continue
hqm_dataframe = pd.DataFrame(hqm_tickers)

,Ticker,Price,Number of shares to buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,139.10,N/A,0.141684,N/A,-0.208403,N/A,-0.137756,N/A,0.065754,N/A,N/A
1,AAL,17.37,N/A,-0.242104,N/A,-0.223993,N/A,-0.085277,N/A,-0.014849,N/A,N/A
2,AAP,217.98,N/A,0.213081,N/A,0.006970,N/A,-0.069404,N/A,0.079016,N/A,N/A
3,AAPL,176.35,N/A,0.474578,N/A,0.194645,N/A,-0.011325,N/A,0.071761,N/A,N/A
4,ABBV,164.05,N/A,0.628289,N/A,0.539818,N/A,0.226621,N/A,0.108198,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,124.80,N/A,0.141819,N/A,-0.038533,N/A,-0.112784,N/A,-0.013958,N/A,N/A
498,ZBH,125.11,N/A,-0.217639,N/A,-0.179663,N/A,-0.022929,N/A,0.005453,N/A,N/A
499,ZBRA,446.32,N/A,-0.081512,N/A,-0.241761,N/A,-0.269297,N/A,0.029311,N/A,N/A
500,ZION,69.97,N/A,0.338428,N/A,0.157180,N/A,0.106693,N/A,0.024182,N/A,N/A


## Calculatiing momentum percentiles

We now need to calculate momentum percentile scores for each stock according to the following metrics:
- One-year price return
- Six-Month Price Return
- Three-Month Price Return
- One-Month Price Return


In [94]:
# from scipy.stats import percentileofscore as score
time_periods = ['One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col],hqm_dataframe.loc[row, change_col])

,Ticker,Price,Number of shares to buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,139.10,N/A,0.141684,48.605578,-0.208403,7.968127,-0.137756,20.119522,0.065754,62.549801,N/A
1,AAL,17.37,N/A,-0.242104,4.780876,-0.223993,6.573705,-0.085277,34.262948,-0.014849,21.513944,N/A
2,AAP,217.98,N/A,0.213081,62.749004,0.006970,47.211155,-0.069404,37.051793,0.079016,68.525896,N/A
3,AAPL,176.35,N/A,0.474578,88.645418,0.194645,81.474104,-0.011325,51.992032,0.071761,65.139442,N/A
4,ABBV,164.05,N/A,0.628289,94.223108,0.539818,95.61753,0.226621,90.438247,0.108198,81.673307,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,124.80,N/A,0.141819,48.804781,-0.038533,37.250996,-0.112784,26.89243,-0.013958,22.310757,N/A
498,ZBH,125.11,N/A,-0.217639,5.776892,-0.179663,10.756972,-0.022929,49.003984,0.005453,33.266932,N/A
499,ZBRA,446.32,N/A,-0.081512,16.932271,-0.241761,5.776892,-0.269297,1.792829,0.029311,44.621514,N/A
500,ZION,69.97,N/A,0.338428,78.087649,0.157180,76.494024,0.106693,79.482072,0.024182,42.231076,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [95]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        percentile_col = f'{time_period} Return Percentile'
        momentum_percentiles.append(hqm_dataframe.loc[row, percentile_col])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

,Ticker,Price,Number of shares to buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,139.10,N/A,0.141684,48.605578,-0.208403,7.968127,-0.137756,20.119522,0.065754,62.549801,34.810757
1,AAL,17.37,N/A,-0.242104,4.780876,-0.223993,6.573705,-0.085277,34.262948,-0.014849,21.513944,16.782869
2,AAP,217.98,N/A,0.213081,62.749004,0.006970,47.211155,-0.069404,37.051793,0.079016,68.525896,53.884462
3,AAPL,176.35,N/A,0.474578,88.645418,0.194645,81.474104,-0.011325,51.992032,0.071761,65.139442,71.812749
4,ABBV,164.05,N/A,0.628289,94.223108,0.539818,95.61753,0.226621,90.438247,0.108198,81.673307,90.488048
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,124.80,N/A,0.141819,48.804781,-0.038533,37.250996,-0.112784,26.89243,-0.013958,22.310757,33.814741
498,ZBH,125.11,N/A,-0.217639,5.776892,-0.179663,10.756972,-0.022929,49.003984,0.005453,33.266932,24.701195
499,ZBRA,446.32,N/A,-0.081512,16.932271,-0.241761,5.776892,-0.269297,1.792829,0.029311,44.621514,17.280876
500,ZION,69.97,N/A,0.338428,78.087649,0.157180,76.494024,0.106693,79.482072,0.024182,42.231076,69.073705


## Selecting the 50 Best Momentum Stocks

In [96]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)

,Ticker,Price,Number of shares to buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MOS,68.91,N/A,1.312730,98.406375,0.951955,98.406375,0.759066,99.003984,0.526385,99.203187,98.75498
1,OXY,60.44,N/A,1.173585,97.410359,1.101482,99.203187,1.052997,99.203187,0.503484,99.003984,98.705179
2,CF,111.00,N/A,1.484617,98.605578,1.002198,98.804781,0.490389,97.609562,0.369520,98.804781,98.456175
3,MRO,26.74,N/A,1.511326,98.804781,1.007858,99.003984,0.606387,98.605578,0.195136,95.61753,98.007968
4,APA,42.68,N/A,1.216570,97.808765,0.905980,98.207171,0.590764,98.207171,0.272059,97.808765,98.007968
5,DVN,64.15,N/A,1.936074,99.003984,0.959566,98.605578,0.497147,98.007968,0.195210,95.816733,97.858566
6,BKR,38.46,N/A,0.811911,96.414343,0.620222,96.414343,0.606231,98.406375,0.347587,98.605578,97.460159
7,HAL,38.18,N/A,0.803043,96.215139,0.825824,97.609562,0.672981,98.804781,0.206201,96.812749,97.360558
8,COP,105.31,N/A,1.049783,97.211155,0.656143,97.211155,0.490753,97.808765,0.196254,96.015936,97.061753
9,CVX,174.40,N/A,0.676877,95.816733,0.716034,97.410359,0.452654,97.011952,0.235395,97.211155,96.86255


## Calculating the number of shares to buy

In [105]:
portofolio_input()

Enter the size of your portofolio: 10000000


In [107]:
position_size = val/ len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe.index)):
    hqm_dataframe.loc[i, 'Number of shares to buy'] = math.floor(position_size/ hqm_dataframe.loc[i, 'Price'])

,Ticker,Price,Number of shares to buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MOS,68.91,2902,1.312730,98.406375,0.951955,98.406375,0.759066,99.003984,0.526385,99.203187,98.75498
1,OXY,60.44,3309,1.173585,97.410359,1.101482,99.203187,1.052997,99.203187,0.503484,99.003984,98.705179
2,CF,111.00,1801,1.484617,98.605578,1.002198,98.804781,0.490389,97.609562,0.369520,98.804781,98.456175
3,MRO,26.74,7479,1.511326,98.804781,1.007858,99.003984,0.606387,98.605578,0.195136,95.61753,98.007968
4,APA,42.68,4686,1.216570,97.808765,0.905980,98.207171,0.590764,98.207171,0.272059,97.808765,98.007968
5,DVN,64.15,3117,1.936074,99.003984,0.959566,98.605578,0.497147,98.007968,0.195210,95.816733,97.858566
6,BKR,38.46,5200,0.811911,96.414343,0.620222,96.414343,0.606231,98.406375,0.347587,98.605578,97.460159
7,HAL,38.18,5238,0.803043,96.215139,0.825824,97.609562,0.672981,98.804781,0.206201,96.812749,97.360558
8,COP,105.31,1899,1.049783,97.211155,0.656143,97.211155,0.490753,97.808765,0.196254,96.015936,97.061753
9,CVX,174.40,1146,0.676877,95.816733,0.716034,97.410359,0.452654,97.011952,0.235395,97.211155,96.86255


## Formatting and Saving Our Excel Output

In [109]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum_Strategy', index = False)
writer.save()